# Analyze Results

After running `scripts/4_compute.py` analyze your results here!

In [17]:
import numpy as np
from pathlib import Path

from qcio import Structure, view, ProgramOutput, align, rmsd, constants

In [18]:
# Constants
DATA_DIR = Path("../data")
STRUCT_DIR = DATA_DIR / "structures"
CALC_DIR = DATA_DIR / "calcs"

## Open and Inspect Results
**Notice that CREST found MANY conformers! This is way too many to inspect by hand AND `py3dmol` (the visualizer) does not get happy past ~20 structures (perhaps they will improve this sometime??)**

In [19]:
u2rc_conf = ProgramOutput.open(CALC_DIR / "u2-rc-crest-conformer_search.json")

In [20]:
print(f"Conformers found: {len(u2rc_conf.results.conformers)}")

Conformers found: 179


### Let's look at a few to get started

In [21]:
# u2rc_conf.results.conformers is a list. If we pass a list of structure view.view will animate
# it for us. Passing * infront of the list unpacks it and passes the list as individual Structures.
view.view(*u2rc_conf.results.conformers[:6])

### We need to start generating statistics on these conformers to understand them. 

Let's generate structures for a binding energy caculation to see how this varies across conformers. These indices are ugly due to how `openbabel` generated the structure. In a dream world the indices are continuous across a fragment!

### Create Fragment Function and Sanity Check It

In [24]:
def generate_fragments(structure: Structure):
    # Generate fragments for the complex
    syms = structure.symbols
    geom = structure.geometry
    # Slide notation (starts_with, up_to_not_including)
    meoh_idxs = [(38, 40), (57, 61)]
    lla_idxs = [(28, 38), (48, 56)]
    cat_idxs = [(0,28), (40, 48)]
    
    meoh = Structure(
        symbols=syms[meoh_idxs[0][0]:meoh_idxs[0][1]] + syms[meoh_idxs[1][0]:meoh_idxs[1][1]], 
        geometry=np.vstack((geom[meoh_idxs[0][0]:meoh_idxs[0][1]], geom[meoh_idxs[1][0]:meoh_idxs[1][1]]))
    )
    lla = Structure(
        symbols=syms[lla_idxs[0][0]:lla_idxs[0][1]] + syms[lla_idxs[1][0]:lla_idxs[1][1]], 
        geometry=np.vstack((geom[lla_idxs[0][0]:lla_idxs[0][1]], geom[lla_idxs[1][0]:lla_idxs[1][1]]))
    )
    cat = Structure(
        symbols=syms[cat_idxs[0][0]:cat_idxs[0][1]] + syms[cat_idxs[1][0]:cat_idxs[1][1]], 
        geometry=np.vstack((geom[cat_idxs[0][0]:cat_idxs[0][1]], geom[cat_idxs[1][0]:cat_idxs[1][1]]))
    )
    return {
        "cat": cat,
        "lla": lla,
        "meoh": meoh,
        "complex": structure
    }

In [25]:
frags = generate_fragments(u2rc_conf.results.conformers[0])

### Looks Good!

In [26]:
view.view(frags['complex'], frags['cat'], frags['lla'], frags['meoh'], titles=["complex", "cat", "lla", "meoh"])

### Generate All the Fragments for out Calculations

In [27]:
frags_dir = STRUCT_DIR / 'frags'
for i, conf in enumerate(u2rc_conf.results.conformers):
    frags = generate_fragments(conf)
    for name, struct in frags.items():
        struct.save(frags_dir / f"{name}-{i}.json")

## Now we're ready for some high throughput calculations!!

Run the `5_compute.py` script to compute energy values for the reaction complex and all fragments.